Preparing a dataset for Basenji training involves a series of design choices.

The input you bring to the pipeline is:
* BigWig coverage tracks
* Genome FASTA file

First, make sure you have an hg19 FASTA file visible. If you have it already, put a symbolic link into the data directory. Otherwise, I have a machine learning friendly simplified version you can download in the next cell.

In [1]:
import os, subprocess

if not os.path.isfile('data/hg19.ml.fa'):
    subprocess.call('curl -o data/hg19.ml.fa https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa', shell=True)
    subprocess.call('curl -o data/hg19.ml.fa.fai https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa.fai', shell=True)                

Next, let's grab a few CAGE datasets from FANTOM5 related to heart biology.

These data were processed by
1. Aligning with Bowtie2 with very sensitive alignment parameters.
2. Distributing multi-mapping reads and estimating genomic coverage with [bam_cov.py](https://github.com/calico/basenji/blob/master/bin/bam_cov.py)

In [2]:
if not os.path.isfile('data/CNhs11760.bw'):
    subprocess.call('curl -o data/CNhs11760.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs11760.bw', shell=True)
    subprocess.call('curl -o data/CNhs12843.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12843.bw', shell=True)
    subprocess.call('curl -o data/CNhs12856.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12856.bw', shell=True)

Then we'll write out these BigWig files and labels to a samples table.

In [3]:
lines = [['index','identifier','file','clip','sum_stat','description']]
lines.append(['0', 'CNhs11760', 'data/CNhs11760.bw', '384', 'sum', 'aorta'])
lines.append(['1', 'CNhs12843', 'data/CNhs12843.bw', '384', 'sum', 'artery'])
lines.append(['2', 'CNhs12856', 'data/CNhs12856.bw', '384', 'sum', 'pulmonic_valve'])

samples_out = open('data/heart_wigs.txt', 'w')
for line in lines:
    print('\t'.join(line), file=samples_out)
samples_out.close()

Next, we want to choose genomic sequences to form batches for stochastic gradient descent, divide them into training/validation/test sets, and construct TFRecords to provide to downstream programs.

The script [basenji_data.py](https://github.com/calico/basenji/blob/master/bin/basenji_data.py) implements this procedure.

The most relevant options here are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -d | 0.1 | Down-sample the genome to 10% to speed things up here. |
| -g | data/unmap_macro.bed | Dodge large-scale unmappable regions like assembly gaps. |
| -l | 131072 | Sequence length. |
| --local | True | Run locally, as opposed to on my SLURM scheduler. |
| -o | data/heart_l131k | Output directory |
| -p | 8 | Uses multiple concourrent processes to read/write. |
| -t | .1 | Hold out 10% sequences for testing. |
| -v | .1 | Hold out 10% sequences for validation. |
| -w | 128 | Pool the nucleotide-resolution values to 128 bp bins. |
| fasta_file| data/hg19.ml.fa | FASTA file to extract sequences from. |
| targets_file | data/heart_wigs.txt | Target samples table with BigWig paths. |

In [4]:
! basenji_data.py -d .1 -g data/unmap_macro.bed -l 131072 --local -o data/heart_l131k -p 8 -t .1 -v .1 -w 128 data/hg19.ml.fa data/heart_wigs.txt

Contigs divided into
 Train:  4701 contigs, 2169074921 nt (0.8005)
 Valid:   572 contigs,  270358978 nt (0.0998)
 Test:    584 contigs,  270330829 nt (0.0998)
/Users/davidkelley/code/Basenji/bin/basenji_data.py:237: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  targets_df = pd.read_table(targets_file, index_col=0)
Skipping existing data/heart_l131k/seqs_cov/0.h5
Skipping existing data/heart_l131k/seqs_cov/1.h5
Skipping existing data/heart_l131k/seqs_cov/2.h5
basenji_data_write.py -s 0 -e 256 data/hg19.ml.fa data/heart_l131k/sequences.bed data/heart_l131k/seqs_cov data/heart_l131k/tfrecords/train-0.tfr &> data/heart_l131k/tfrecords/train-0.err
basenji_data_write.py -s 256 -e 512 data/hg19.ml.fa data/heart_l131k/sequences.bed data/heart_l131k/seqs_cov data/heart_l131k/tfrecords/train-1.tfr &> data/heart_l131k/tfrecords/train-1.err
basenji_data_write.py -s 512 -e 584 data/hg19.ml.fa data/heart_l131k/sequences.bed data/heart_l131k/seqs_cov data/heart_l1

Now, data/heart_l131k contains relevant data for training.

*contigs.bed* contains the original large contiguous regions from which training sequences were taken (possibly strided).
*sequences.bed*

contains the train/valid/test sequences.

In [5]:
! cut -f4 data/heart_l131k/sequences.bed | sort | uniq -c

  37 test
 485 train
  63 valid


In [6]:
! head -n3 data/heart_l131k/sequences.bed

chr1	117100811	117231883	train
chrX	37894721	38025793	train
chr7	152476606	152607678	train


In [7]:
! grep valid data/heart_l131k/sequences.bed | head -n3

chr10	66147720	66278792	valid
chr4	20166618	20297690	valid
chr6	103944678	104075750	valid


In [8]:
! grep test data/heart_l131k/sequences.bed | head -n3

chr5	151856657	151987729	test
chr2	88586481	88717553	test
chr13	89262763	89393835	test


In [9]:
! ls -l data/heart_l131k/tfrecords/*.tfr

-rw-r--r--  1 davidkelley  staff  15148824 May 21 16:04 data/heart_l131k/tfrecords/test-0.tfr
-rw-r--r--  1 davidkelley  staff  15175600 May 21 16:04 data/heart_l131k/tfrecords/test-1.tfr
-rw-r--r--  1 davidkelley  staff   3194851 May 21 16:04 data/heart_l131k/tfrecords/test-2.tfr
-rw-r--r--  1 davidkelley  staff  15149599 May 21 16:04 data/heart_l131k/tfrecords/train-0.tfr
-rw-r--r--  1 davidkelley  staff  15173521 May 21 16:04 data/heart_l131k/tfrecords/train-1.tfr
-rw-r--r--  1 davidkelley  staff   4264786 May 21 16:03 data/heart_l131k/tfrecords/train-2.tfr
-rw-r--r--  1 davidkelley  staff  15151746 May 21 16:04 data/heart_l131k/tfrecords/valid-0.tfr
-rw-r--r--  1 davidkelley  staff  15177910 May 21 16:04 data/heart_l131k/tfrecords/valid-1.tfr
-rw-r--r--  1 davidkelley  staff   3618164 May 21 16:03 data/heart_l131k/tfrecords/valid-2.tfr
